In [2]:
# pip install datasets 
# pip install pinecone-client
# pip install --no-index torch-scatter -f https://pytorch-geometric.com/whl/torch-1.8.0+cpu.html
# pip install sentence-transformers

from datasets import load_dataset
import pandas as pd

table0 = pd.read_excel("MS_IS_all_modules_orginal_15_0.xlsx")
table0 = table0.astype(str)
table1 = pd.read_excel("MS_IS_all_modules_orginal_15_1.xlsx")
table1 = table1.astype(str)
table2 = pd.read_excel("MS_IS_all_modules_orginal_15_2.xlsx")  
table2 = table2.astype(str)

# store all tables in the tables list
tables = []
tables.append(table0)
tables.append(table1)
tables.append(table2)



In [3]:
tables[2]

,Module title,Abbreviation,Module coordinator,Module offered by,ETCS,Method of grading,Duration,Module level,Contents,Intended learning outcomes,Courses,Method of assessment,Allocation of places,Additional information,Workload,Teaching cycle,Referred to in LPO I
0,Digital Entrepreneurship,12-M-UGF3-182-m01,Faculty of Business Management and Economics,Holder of the Chair of Entrepreneurship and St...,5,numerical grade,1 semester,graduate,This module provides an introduction into digi...,Educational aims: Clarify the role of digital ...,V (2) + Ü (2)Module taught in: English,a) written examination (approx. 60 to 120 minu...,--,--,150 h,--,--
1,Advanced Seminar: Entrepreneurship and Management,12-M-SAS-182-m01,Faculty of Business Management and Economics,Holder of the Chair of Entrepreneurship and St...,10,numerical grade,1 semester,graduate,Students develop seminar papers on varying top...,Educational aims• Enable students to position ...,S (2),term paper (approx. 20 pages) and presentation...,20 places. (1) Should the number of applicatio...,--,300 h,--,--
2,Global Logistics & Supply Chain Management,12-M-GLSC-182-m01,Faculty of Business Management and Economics,Holder of the Chair of Logistics and Quantitat...,5,numerical grade,1 semester,graduate,"The course ""Global Logistics & Supply Chain Ma...",After completing this course students can(i) a...,V (2) + Ü (2)Module taught in: English,a) written examination (approx. 60 minutes) or...,--,--,150 h,--,--
3,Advanced Operations & Logistics Management,12-M-AOLM-182-m01,Faculty of Business Management and Economics,Holder of the Chair of Logistics and Quantitat...,5,numerical grade,1 semester,graduate,"The course ""Advanced Operations & Logistics Ma...",After completing this course students can(i) a...,V (2) + Ü (2)Module taught in: English,a) written examination (approx. 60 minutes) or...,--,--,150 h,--,--
4,Seminar: Operations Management,12-M-SN-161-m01,Faculty of Business Management and Economics,Holder of the Chair of Business Management and...,10,numerical grade,1 semester,graduate,"With the help of topics from the area of ""Oper...",Students will learn how to convince a critical...,S (2),term paper (approx. 20 to 25 pages) and presen...,--,--,300 h,--,--
5,Adaption and Continuous System Engineering,12-ACSE-161-m01,Faculty of Business Management and Economics,holder of the Chair of Business Management and...,5,numerical grade,1 semester,graduate,Business Suite: The constantly changing enviro...,Business Suite: Students learn about the vario...,V (2) + Ü (2),a) written examination (approx. 60 minutes) or...,20 places. (1) Should the number of applicatio...,--,150 h,--,--
6,Business Service Platforms 2,12-AGP2-192-m01,Faculty of Business Management and Economics,Holder of the Chair of Business Management and...,5,numerical grade,1 semester,graduate,The next generation of business service platfo...,Be aware of the growing economic importance of...,V (2),Written examination (approx. 60 minutes)Credit...,40 places.Should the number of applications ex...,--,150 h,--,--_x000C_
7,Business Service Platforms 1,12-BSA-192-m01,Faculty of Business Management and Economics,Holder of the Chair of Business Management and...,5,numerical grade,1 semester,graduate,A next generation of enterprise systems called...,Be aware of the big business productivity prog...,V (2),Written examination (approx. 60 minutes)Credit...,40 places.Should the number of applications ex...,--,150 h,--,--
8,"Business Processes Organisation, Business Soft...",12-GLP-161-m01,Faculty of Business Management and Economics,holder of the Chair of Business Management and...,5,numerical grade,1 semester,graduate,ERP systems have become key elements of succes...,"After completing this module, students should ...",V (2) + Ü (2),written examination (approx. 60 minutes)Langua...,20 places. (1) Should the number of applicatio...,--,150 h,--,--
9,Work and Information,12-ITA-161-m01,Faculty of Business Management and Economics,holder of

In [4]:
import torch
from sentence_transformers import SentenceTransformer

# set device to GPU if available
device = 'cuda' if torch.cuda.is_available() else 'cpu'
# load the table embedding model from huggingface models hub
retriever = SentenceTransformer("deepset/all-mpnet-base-v2-table", device=device)
retriever




SentenceTransformer(
  (0): Transformer({'max_seq_length': 384, 'do_lower_case': False}) with Transformer model: MPNetModel 
  (1): Pooling({'word_embedding_dimension': 768, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False})
  (2): Normalize()
)

In [5]:
def _preprocess_tables(tables: list):
    processed = []
    # loop through all tables
    for table in tables:
        # convert the table to csv and 
        processed_table = "\n".join([table.to_csv(index=False)])
        # add the processed table to processed list
        processed.append(processed_table)
    return processed




### The formatted table may not make sense to us, but the embedding model is trained to understand it and generate accurate embeddings.

In [6]:
# format all the dataframes in the tables list
processed_tables = _preprocess_tables(tables)
# display the formatted table
processed_tables[2]

'Module title,Abbreviation,Module coordinator,Module offered by,ETCS,Method of grading,Duration,Module level,Contents,Intended learning outcomes,Courses,Method of assessment,Allocation of places,Additional information,Workload,Teaching cycle,Referred to in LPO I\r\nDigital Entrepreneurship,12-M-UGF3-182-m01,Faculty of Business Management and Economics,Holder of the Chair of Entrepreneurship and Strategy,5,numerical grade,1 semester,graduate,This module provides an introduction into digital entrepreneurship and digital transformation. (1) Introduction (2) Digital business models (3) Identifying and exploiting opportunities for digital entrepreneurship (4) Strategies for creating competitive advantage in digital entrepreneurship (5) Digital marketing for entrepreneurs (6) Crowd-funding for entrepreneurs (7) Design thinking (8) Lean startup (9) Platform ecosystems and online communities (10) Digital strategy and digital transformation (11) The agile organization (12) Crowdsourcing (13) Cy

In [7]:
import pinecone

# connect to pinecone environment
pinecone.init(
    api_key="1d3ebed2-b4fa-4523-9cd4-722ac005fc6b",
    environment="us-west1-gcp-free"
)

In [11]:
# you can choose any name for the index
index_name = "table-qa-module-catalogue-split"

# check if the table-qa index exists
if index_name not in pinecone.list_indexes():
    # create the index if it does not exist
    pinecone.create_index(
        index_name,
        dimension=768,
        metric="cosine"
    )

# connect to table-qa index we created
index = pinecone.Index(index_name)

In [12]:
from tqdm.auto import tqdm

# we will use batches of 64
batch_size = 64

for i in tqdm(range(0, len(processed_tables), batch_size)):
    # find end of batch
    i_end = min(i+batch_size, len(processed_tables))
    # extract batch
    batch = processed_tables[i:i_end]
    # generate embeddings for batch
    emb = retriever.encode(batch).tolist()
    # create unique IDs ranging from zero to the total number of tables in the dataset
    ids = [f"{idx}" for idx in range(i, i_end)]
    # add all to upsert list
    to_upsert = list(zip(ids, emb))
    # upsert/insert these records to pinecone
    _ = index.upsert(vectors=to_upsert)

# check that we have all vectors in index
index.describe_index_stats()

  0%|          | 0/1 [00:00<?, ?it/s]

{'dimension': 768,
 'index_fullness': 0.0,
 'namespaces': {'': {'vector_count': 3}},
 'total_vector_count': 3}

In [28]:
query = "What is the module title 'IT Management' about?"
# generate embedding for the query
xq = retriever.encode([query]).tolist()
# query pinecone index to find the table containing answer to the query
result = index.query(xq, top_k=3)
result


{'matches': [{'id': '0', 'score': 0.707598925, 'values': []},
             {'id': '2', 'score': 0.603760898, 'values': []},
             {'id': '1', 'score': 0.551980793, 'values': []}],
 'namespace': ''}

In [16]:
id = int(result["matches"][0]["id"])
tables[id].head()

,Module title,Abbreviation,Module coordinator,Module offered by,ETCS,Method of grading,Duration,Module level,Contents,Intended learning outcomes,Courses,Method of assessment,Allocation of places,Additional information,Workload,Teaching cycle,Referred to in LPO I
0,Information Processing within Organizations,12-IV-161-m01,Faculty of Business Management and Economics,holder of the Chair of Business Management and...,5,numerical grade,1 semester,graduate,\n\nContent:\nThis course provides students wi...,"After completing the course ""Integrated Inform...",\n\nV (2) + Ü (2)\n\n,\n\nwritten examination (approx. 60 minutes)\n...,\n\n--\n\n,\n\n--\n\n,\n\n150 h\n\n,\n\n--\n\n,\n\n--\n\n
1,IT-Management,12-M-ITM-161-m01,Faculty of Business Management and Economics,Holder of the Chair of Information Systems Eng...,5,numerical grade,1 semester,graduate,\n\nContent:\nThis course provides students wi...,"After completing the course ""IT Management"", s...",\n\nV (2) + Ü (2)\n\n,\n\na) written examination (approx. 60 minutes...,\n\n--\n\n,\n\n--\n\n,\n\n150 h\n\n,\n\n--\n\n,\n\n--\n\n
2,Project Seminar,12-PS-192-m01,Faculty of Business Management and Economics,Holder of the Chair of Business Management and...,15,numerical grade,1 semester,graduate,\n\nContent:\nIn small project teams of 4 to 1...,"After completing the course ""Projektseminar"", ...",\n\nS (2)\n\n,\n\nproject: preparing a conceptual design (ap...,\n\n--\n\n,\n\n--\n\n,\n\n450 h\n\n,\n\n--\n\n,\n\n--\n\n
3,Information Retrieval,10-I=IR-161-m01,Institute of Computer Science,Dean of Studies Informatik (Computer Science),5,numerical grade,1 semester,graduate,\n\nIR models (e. g. Boolean and vector space ...,The students possess theoretical and practical...,\n\nV (2) + Ü (2)\n\n,\n\nwritten examination (approx. 60 to 120 min...,\n\n--\n\n,\n\nFocuses available for students of the Mast...,\n\n150 h\n\n,\n\n--\n\n,\n\n--\n\n
4,Analysis and Design of Programs,10-I=PA-161-m01,Institute of Computer Science,holder of the Chair of Computer Science II,5,numerical grade,1 semester,graduate,"\n\nProgram analysis, model creation in softwa...","The students are able to analyse programs, to ...",\n\nV (2) + Ü (2)\n\n,\n\nwritten examination (approx. 60 to 120 min...,\n\n--\n\n,\n\nFocuses available for students of the Mast...,\n\n150 h\n\n,\n\n--\n\n,\n\n--\n\n


#### The table returned by the Pinecone index indeed has the answer to our query. Now we need a model that can read this table and extract the precise answer.

In [17]:
from transformers import pipeline, TapasTokenizer, TapasForQuestionAnswering

model_name = "google/tapas-base-finetuned-wtq"
# load the tokenizer and the model from huggingface model hub
tokenizer = TapasTokenizer.from_pretrained(model_name)
model = TapasForQuestionAnswering.from_pretrained(model_name, local_files_only=False)
# load the model and tokenizer into a question-answering pipeline
pipe = pipeline("table-question-answering",  model=model, tokenizer=tokenizer, device=device)

In [18]:
pipe(table=tables[id], query=query)

{'answer': 'AVERAGE > 5',
 'coordinates': [(1, 4)],
 'cells': ['5'],
 'aggregator': 'AVERAGE'}

In [19]:
def query_pinecone(query):
    # generate embedding for the query
    xq = retriever.encode([query]).tolist()
    # query pinecone index to find the table containing answer to the query
    result = index.query(xq, top_k=1)
    # return the relevant table from the tables list
    return tables[int(result["matches"][0]["id"])]

In [20]:
def get_answer_from_table(table, query):
    # run the table and query through the question-answering pipeline
    answers = pipe(table=table, query=query)
    return answers

In [22]:
query = "Which modules are about IT?"
table = query_pinecone(query)

In [23]:
get_answer_from_table(table, query)

{'answer': 'AVERAGE > 5',
 'coordinates': [(1, 4)],
 'cells': ['5'],
 'aggregator': 'AVERAGE'}